- 소민: 2020년 7월 ~ 2020년 12월

In [1]:
import pandas as pd
import numpy as np

from pprint import pprint
from tqdm.notebook import tqdm

df = pd.read_csv("data/total_data.csv")

## Filtering by date

In [2]:
df.head(2)

,날짜,제목,내용,url,text_prep,tokenized_text,jeju_place,jeju_place_google,jeju_place_google_str
0,2020-01-01 00:00:00,"제주여행의 시작 : 국립제주박물관 (제주 가볼만한곳, 제주 유배인의 이야기, 비 오...",제주여행의 시작\n국립제주박물관\n박물관 도장깨기!이번 저의 제주도 여행 테마는 ‘...,https://blog.naver.com/tnrwlschl/221753997486,제주여행의 시작 국립제주박물관 박물관 도장깨기 이번 저의 제주도 여행 테마는 다크...,"['제주', '여행의', '시작', '국립제주박물관', '박물관', '도장깨기', ...","['국립제주박물관', '주차장', '국립제주박물관', '주차장', '대륙', '대해...",[],NaN
1,2020-01-01 01:15:00,"2019 제주여행 3 :: 성산일출봉, 풍미 독서, 세화해수욕장, 모이소",2박 3일 제주여행의 마지막 날 아침\n성산 일출봉을 향했다.입장 시간이 이미 해가...,https://blog.naver.com/chersy/221755888807,박 일 제주여행의 마지막 날 아침 성산 일출봉을 향했다.입장 시간이 이미 해가 ...,"['제주', '여행의', '마지막', '아침', '성산', '일출봉을', '향했다....","['제일', '우리', '하나', '돌하르방', '성산일출봉', '돌하르방', '마...",['성산일출봉'],성산일출봉


In [3]:
df['날짜'] = pd.to_datetime(df['날짜'])
df.shape

(69513, 9)

In [4]:
df_filtered = df.loc[df["날짜"].between('2020-07-01', '2020-12-31')]
df_filtered.shape

(21116, 9)

In [5]:
df_filtered = df_filtered.dropna()
df_filtered.shape

(8410, 9)

In [6]:
df_filtered.to_csv("data/total_data_filtered.csv", index=False)

## Split by places

In [7]:
df_filtered = pd.read_csv("data/total_data_filtered.csv")
df_filtered.shape

(8410, 9)

In [8]:
df_filtered.head(2)

,날짜,제목,내용,url,text_prep,tokenized_text,jeju_place,jeju_place_google,jeju_place_google_str
0,2020-07-01 00:11:00,20200622-0624제주여행/셋째날,셋째날은 비가 온다는걸 미리 알고있었기에\n별다른 일정을 잡지 않았다.\n일출은 사...,https://blog.naver.com/mylovemiso/222017317029,셋째날은 비가 온다는걸 미리 알고있었기에 별다른 일정을 잡지 않았다. 일출은 사치 ...,"['셋째날', '비가', '온다', '는걸', '미리', '알고', '있었기에', ...","['산방산', '중문', '산방산', '산방산', '중문', '산방산', '제일',...",['산방산'],산방산
1,2020-07-01 00:32:00,"제주여행 #3 : 성산 @애월 텐동 맛집 료리아, 성산여행지 보롬왓 & 빛의 벙커...","2020.05.02 ~ 05\n제주여행 3일차,\n아침은 라면으로 해장을 했는데\n...",https://blog.naver.com/linzyworld/222017333868,. . 제주여행 일차 아침은 라면으로 해장을 했는데 퇴실하...,"['제주', '여행', '일차', '아침', '라면', '으로', '해장을', '했...","['주차장', '주차장', '우리', '제일', '넓은', '한잔', '농장', '...",['빛의벙커'],빛의벙커


In [9]:
df_filtered['jeju_place_google'] = [place.split(', ') for place in df_filtered['jeju_place_google_str'] ]

In [10]:
type(df_filtered['jeju_place_google'].iloc[0])

list

In [11]:
pprint(df_filtered.iloc[1].jeju_place_google)

['빛의벙커']


In [12]:
import re

s_idx=0
result_list = []
for idx, row in tqdm(df_filtered.iterrows()):
    jeju_place_google = row['jeju_place_google']
#     print(len(jeju_place_google))
    if len(jeju_place_google) == 1:
        result_list.append([row['날짜'], row['text_prep'], row['jeju_place_google_str'], idx])
        continue
    sents = re.split(r'[.!?]', row['text_prep'])
    target_place = jeju_place_google[0]
    for sent_idx, s in enumerate(sents):        
        if target_place in s:
            s_idx = sent_idx
        for place in jeju_place_google:
            if (target_place != place) & (place in s):
#                 print(target_place)
#                 print(s_idx, sent_idx)
#                 print("".join(sents[s_idx:sent_idx+1]))
                result_list.append([row['날짜'], "".join(sents[s_idx:sent_idx+1]), target_place, idx])
                target_place = place
                s_idx = sent_idx

result_list.append([row['날짜'], "".join(sents[s_idx:]), target_place, idx])

0it [00:00, ?it/s]

In [13]:
splited_data = pd.DataFrame(result_list, columns=["date", 'text', 'place', 'idx'])

In [14]:
splited_data

,date,text,place,idx
0,2020-07-01 00:11:00,셋째날은 비가 온다는걸 미리 알고있었기에 별다른 일정을 잡지 않았다. 일출은 사치 ...,산방산,0
1,2020-07-01 00:32:00,. . 제주여행 일차 아침은 라면으로 해장을 했는데 퇴실하...,빛의벙커,1
2,2020-07-01 01:01:00,비가 하루종일 올거란 일기예보와는 달리 쾌창하다못해 구름한점없는 아침이였다.일어나자...,우도,2
3,2020-07-01 02:32:00,가다가 우도 비양도를 발견하고 바로 들어갔음,우도,3
4,2020-07-01 02:32:00,가다가 우도 비양도를 발견하고 바로 들어갔음 보민이가 거의 바닥에 붙어서 사진을 찍...,비양도,3
...,...,...,...,...
12873,2020-12-30 23:31:00,첫 여행지는 함덕해수욕장 금강산도 식후경 갈치조림 먹으러 고고링건너편 주차...,함덕해수욕장,8409
12874,2020-12-30 23:31:00,다랑쉬오름은 비온뒤라 오르기 힘들꺼같아서 포기 스을카페는 너무 가고싶어서 사장님...,빛의벙커,8409
12875,2020-12-30 23:31:00,다랑쉬오름은 비온뒤라 오르기 힘들꺼같아서 포기 스을카페는 너무 가고싶어서 사장님...,성산일출봉,8409
12876,2020-12-30 23:31:00,일차 진에어 제주공항 스타렌트카 함덕해수욕장 테우 오드랑베이커리 스을 만석...,함덕해수욕장,8409


In [15]:
splited_data.to_csv("splited_data.csv", index=False)